In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from ntv_numpy import Xdataset
import ntv_pandas as npd
import cbor2


# multidimensional

In [2]:
xdss = [xr.Dataset({"var": (['a', 'b', 'c', 'd', 'e'], np.arange(100000).reshape(10, 10, 10, 10, 10))},
                    coords={"a": np.arange(10), "b": np.arange(10, 20), "c": np.arange(20, 30), "d": np.arange(30, 40), "e": np.arange(40, 50) }),
        xr.Dataset({"var": (['a', 'b', 'c', 'd', 'e'], np.arange(100000).reshape(10, 10, 10, 10, 10))},
                    coords={"a": np.array(['sdfghjsdfghj', 'dfghjkdfghjk', 'azertyazerty', 'zertyuzertyu', 'rtyuiortyuio', 
                                           'uiopqsuiopqs', 'qsdfghqsdfgh', 'ghjklmghjklm', 'wxcvbnwxcvbn', 'hjklmwhjklmw']),
                            "b": np.array(['uiopqsuiopqs', 'qsdfghqsdfgh', 'azertyazerty', 'zertyuzertyu', 'rtyuiortyuio',
                                           'sdfghjsdfghj', 'dfghjkdfghjk', 'wxcvbnwxcvbn', 'ghjklmghjklm', 'hjklmwhjklmw']),
                            "c": np.array(['dfghjkdfghjk', 'ghjklmghjklm', 'azertyazerty', 'uiopqsuiopqs', 'qsdfghqsdfgh', 
                                           'sdfghjsdfghj', 'wxcvbnwxcvbn', 'hjklmwhjklmw', 'zertyuzertyu', 'rtyuiortyuio']),
                            "d": np.array(['azertyazerty', 'zertyuzertyu', 'rtyuiortyuio', 'uiopqsuiopqs', 'qsdfghqsdfgh', 
                                           'sdfghjsdfghj', 'dfghjkdfghjk', 'ghjklmghjklm', 'wxcvbnwxcvbn', 'hjklmwhjklmw']),
                            "e": np.array(['qsdfghqsdfgh', 'sdfghjsdfghj', 'dfghjkdfghjk', 'ghjklmghjklm', 'wxcvbnwxcvbn',
                                           'hjklmwhjklmw', 'azertyazerty', 'zertyuzertyu', 'rtyuiortyuio', 'uiopqsuiopqs'])})
]


In [3]:
res = []
for xds in xdss:
    xnd = Xdataset.from_xarray(xds)
    df_multi = xnd.to_dataframe(json_name=False, info=False).reset_index().sample(frac=1)
    csv = df_multi.to_csv()
    df_multi.to_csv('test_multi.csv')

    xnd = Xdataset.from_dataframe(df_multi)
    jsn_txt = xnd.to_json(notype=[True]*len(xnd), encoded=True)
    jsn = xnd.to_json(notype=[True]*len(xnd))
    #jsn = xnd.to_json(notype=[True]*len(xnd), format=['complete']*len(xnd))
    jsb = cbor2.dumps(jsn)
    
    res.append([len(jsb)/1024, len(jsn_txt)/1024, len(csv)/1024])
df_multi

,a:string,b:string,c:string,d:string,e:string,var:int32
19102,dfghjkdfghjk,hjklmwhjklmw,ghjklmghjklm,azertyazerty,dfghjkdfghjk,19102
70124,ghjklmghjklm,uiopqsuiopqs,ghjklmghjklm,rtyuiortyuio,wxcvbnwxcvbn,70124
29147,azertyazerty,hjklmwhjklmw,ghjklmghjklm,qsdfghqsdfgh,zertyuzertyu,29147
96066,hjklmwhjklmw,dfghjkdfghjk,dfghjkdfghjk,dfghjkdfghjk,azertyazerty,96066
20312,azertyazerty,uiopqsuiopqs,uiopqsuiopqs,zertyuzertyu,dfghjkdfghjk,20312
...,...,...,...,...,...,...
33945,zertyuzertyu,zertyuzertyu,rtyuiortyuio,qsdfghqsdfgh,hjklmwhjklmw,33945
37691,zertyuzertyu,wxcvbnwxcvbn,wxcvbnwxcvbn,hjklmwhjklmw,sdfghjsdfghj,37691
57752,uiopqsuiopqs,wxcvbnwxcvbn,hjklmwhjklmw,sdfghjsdfghj,dfghjkdfghjk,57752
37330,zertyuzertyu,wxcvbnwxcvbn,uiopqsuiopqs,uiopqsuiopqs,qsdfghqsdfgh,37330


In [4]:
res[0].append(1985.0)
res[1].append(1986.0)
res

[[360.1748046875, 673.08203125, 2615.0703125, 1985.0],
 [360.740234375, 673.6826171875, 7595.5439453125, 1986.0]]

In [5]:
res

[[360.1748046875, 673.08203125, 2615.0703125, 1985.0],
 [360.740234375, 673.6826171875, 7595.5439453125, 1986.0]]

# tree

In [6]:
strg = np.arange(2**2)
strg = np.frompyfunc(str, 1, 1)(strg)
strg

array(['0', '1', '2', '3'], dtype=object)

In [7]:
# size = 15
size = 10
arr = np.arange(2**size)
data_int = [arr]
data_str = [np.frompyfunc(str, 1, 1)(arr)]
for i in range(size):
    arr = arr // 2
    data_int.append(arr)
    data_str.append(np.frompyfunc(str, 1, 1)(arr))
tab_int_tree = {'col_' + str(i): data_int[i] for i in range(len(data_int))}
tab_str_tree = {'col_' + str(i): data_str[i] for i in range(len(data_str))}

In [8]:
df_int_tree = pd.DataFrame(tab_int_tree)
df_str_tree = pd.DataFrame(tab_str_tree)
df_int_tree

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10
0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0
2,2,1,0,0,0,0,0,0,0,0,0
3,3,1,0,0,0,0,0,0,0,0,0
4,4,2,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1019,1019,509,254,127,63,31,15,7,3,1,0
1020,1020,510,255,127,63,31,15,7,3,1,0
1021,1021,510,255,127,63,31,15,7,3,1,0
1022,1022,511,255,127,63,31,15,7,3,1,0


In [9]:
df_int_tree = df_int_tree.sample(frac=1).reset_index(drop=True)
df_str_tree = df_str_tree.sample(frac=1).reset_index(drop=True)

In [10]:
res_tree = []
for df in [df_int_tree, df_str_tree]:
    csv = df.to_csv()
    xnd = Xdataset.from_dataframe(df)
    jsn_full_txt = xnd.to_json(notype='all', encoded=True)
    jsn_full = xnd.to_json(notype='all')
    forma = {name:'complete' for name in xnd.names[1:]}
    jsn_comp_txt = xnd.to_json(notype='all', format=forma, encoded=True)
    jsn_comp = xnd.to_json(notype='all', format=forma)
    jsb_comp = cbor2.dumps(jsn_comp)
    
    res_tree.append([len(jsb_comp)/1024, len(jsn_comp_txt)/1024, len(jsn_full_txt)/1024, len(csv)/1024])
res_tree

[[9.2216796875, 18.7119140625, 38.5732421875, 35.3017578125],
 [11.81640625, 22.7099609375, 58.5751953125, 35.3017578125]]

In [11]:
from time import time

df = df_int_tree
tim = [] 
tim.append(time())
csv = df.to_csv()
tim.append(time())
xnd = Xdataset.from_dataframe(df, distr=False)
tim.append(time())
jsn_full_txt = xnd.to_json(notype='all', encoded=True)
tim.append(time())
jsn_full = xnd.to_json(notype='all')
tim.append(time())
forma = {name:'complete' for name in xnd.names[1:]}
tim.append(time())
jsn_comp_txt = xnd.to_json(notype='all', format=forma, encoded=True)
tim.append(time())
jsn_comp = xnd.to_json(notype='all', format=forma)
tim.append(time())
jsb_comp = cbor2.dumps(jsn_comp)
tim.append(time())

dt = [tim[i+1] - tim[i] for i in range(len(tim)-1)]
dt
    

[0.002985239028930664,
 0.513298749923706,
 0.002986431121826172,
 0.002986907958984375,
 0.0,
 0.004979610443115234,
 0.004479885101318359,
 0.0004985332489013672]

In [12]:
from xconnector import PandasConnec
t0=time()
PandasConnec._ximport_analysis(df_int_tree, None, distr=True)
print(time()-t0)

0.5162839889526367
